# 水泵模型 (Pump Model)

`水泵 (Pump)` 类代表水系统中的一个可控水泵。它可以被开启或关闭以提供特定的流量，前提是它能克服上游和下游点之间的水头差。本Jupyter Notebook解释了`Pump`模型，并演示了其在仿真中的使用。

## 状态变量

水泵的状态由以下变量描述：

- `status` (int): 水泵的运行状态 (0为关闭, 1为开启)。
- `outflow` (float): 水泵产生的流量 (m³/s)。
- `power_draw_kw` (float): 水泵消耗的电功率 (kW)。

## 参数

水泵的操作特性由以下参数定义：

- `max_head` (float): 水泵可以克服的最大压头 (m)。
- `max_flow_rate` (float): 当水泵开启且水头在限制范围内时的流量 (m³/s)。
- `power_consumption_kw` (float): 水泵运行时消耗的功率 (kW)。

## 仿真示例

以下示例模拟了水泵的操作。在仿真过程中，水泵被开启和关闭，并定义了上游和下游的水位。仿真跟踪了水泵的状态、出流量和功耗，然后将这些数据绘制成图表。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` (`pip install matplotlib`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.pump import Pump
from swp.core.interfaces import State, Parameters

# 水泵的初始状态和参数
initial_state = State(status=0, outflow=0.0, power_draw_kw=0.0)
parameters = Parameters(
    max_head=25.0,             # 米
    max_flow_rate=2.0,         # m³/s
    power_consumption_kw=150.0 # kW
)

# 创建一个Pump实例
pump = Pump(name="main_pump", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 1  # 时间步长（秒）
simulation_duration = 100
num_steps = int(simulation_duration / dt)

# 存储结果用于绘图
history = []

# 定义外部条件
upstream_head = 10.0
downstream_head = 30.0

# 运行仿真循环
for t in range(num_steps):
    # 在一段时间内开启水泵，然后关闭
    if 20 <= t < 80:
        control_signal = 1  # 开启
    else:
        control_signal = 0  # 关闭
    
    action = {
        'control_signal': control_signal,
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = pump.step(action=action, dt=dt)
    history.append(current_state.copy())

# 打印最终状态
print("最终状态:", pump.get_state())

# 准备绘图数据
time_steps = [i * dt for i in range(num_steps)]
statuses = [s['status'] for s in history]
outflows = [s['outflow'] for s in history]
power_draws = [s['power_draw_kw'] for s in history]

# 绘制结果
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

ax1.plot(time_steps, statuses, label='水泵状态 (Pump Status)', drawstyle='steps-post')
ax1.set_ylabel('状态 (0=关, 1=开)')
ax1.set_title('水泵操作仿真')
ax1.grid(True)
ax1.legend()

ax2.plot(time_steps, outflows, label='出流量 (Outflow)', color='orange')
ax2.set_ylabel('出流量 (m³/s)')
ax2.grid(True)
ax2.legend()

ax3.plot(time_steps, power_draws, label='功耗 (Power Draw)', color='green')
ax3.set_xlabel('时间 (秒)')
ax3.set_ylabel('功率 (kW)')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()
